## 0. Install and Import Dependencies

In [ ]:
import cv2
from matplotlib import pyplot as plt
import numpy as np
import imutils
import cv2
import numpy as np
from skimage import io
import matplotlib.pyplot as plt
from commonfunctions import *

import skimage.io as io
import matplotlib.pyplot as plt
from skimage.color import rgb2gray
from skimage.measure import find_contours
from skimage.draw import rectangle

# Show the figures / plots inside the notebook
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
def preprocess (input_image):
    gray_image = cv2.cvtColor(input_image, cv2.COLOR_BGR2GRAY)
    show_images([gray_image])
    filtered_image = cv2.bilateralFilter(gray_image, 11, 17, 17) 
    show_images([filtered_image])
    after_edge_detection = cv2.Canny(filtered_image, 30, 200) 
    show_images([after_edge_detection])
    element = cv2.getStructuringElement( shape=cv2.MORPH_RECT, ksize=(1, 1))
    cv2.morphologyEx(src=after_edge_detection, op=cv2.MORPH_CLOSE,   kernel=element, dst=after_edge_detection)
    show_images([after_edge_detection])
    return after_edge_detection

# grab_contours

In [ ]:
# def grab_contours(cnts):
#     # if the length the contours tuple returned by cv2.findContours
#     # is '2' then we are using either OpenCV v2.4, v4-beta, or
#     # v4-official
#     if len(cnts) == 2:
#         cnts = cnts[0]

#     # if the length of the contours tuple is '3' then we are using
#     # either OpenCV v3, v4-pre, or v4-alpha
#     elif len(cnts) == 3:
#         cnts = cnts[1]

#     # otherwise OpenCV has changed their cv2.findContours return
#     # signature yet again and I have no idea WTH is going on
#     else:
#         raise Exception(("Contours tuple must have length 2 or 3, "
#             "otherwise OpenCV changed their cv2.findContours return "
#             "signature yet again. Refer to OpenCV's documentation "
#             "in that case"))

#     # return the actual contours array
#     return cnts

# Find contours

In [ ]:
def find_contours(after_edge_detection):
    contours = cv2.findContours(after_edge_detection.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours = imutils.grab_contours(contours)
    contours = sorted(contours, key=cv2.contourArea, reverse=True)
    return contours

In [ ]:
def checkIfPlate(contour, img):
    
    minRatio = 2.3
    maxRatio = 5

    height, width, _ = img.shape 
    area = height * width
    
    minArea = 0.01 * area
    maxArea = 0.25 * area
    
    x,y,w,h = cv2.boundingRect(contour)  
    contourArea = w*h

    if (w/h >  minRatio and  w/h <maxRatio and contourArea > minArea and  contourArea < maxArea ):

        return True
    return False

In [ ]:
def find_plate(input_image):
    after_edge_detection = preprocess(input_image)
    contours = find_contours(after_edge_detection)
    location = None
    candidate_contours = []
    images = []
    for contour in contours:
        perimeter = cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, 0.02*perimeter , True)
        # approx = cv2.approxPolyDP(contour, 10, True)
        if len(approx) == 4:
            location = approx 
            if (checkIfPlate(contour, input_image)): 
                candidate_contours.append(contour)               
                mask = np.zeros(cv2.cvtColor(input_image, cv2.COLOR_BGR2GRAY).shape, np.uint8)
                new_image = cv2.drawContours(mask, [location], 0,255, -1)
                new_image = cv2.bitwise_and(input_image, input_image, mask=mask)
                show_images([new_image])
                (x,y) = np.where(mask==255)
                (x1, y1) = (np.min(x), np.min(y))
                (x2, y2) = (np.max(x), np.max(y))
                images.append(cv2.cvtColor(input_image, cv2.COLOR_BGR2GRAY)[x1:x2+1, y1:y2+1])
    show_images(images)
    if (len(candidate_contours) > 1):
        candidate_contours = sorted(candidate_contours, key=cv2.contourArea, reverse=True)
        perimeter = cv2.arcLength(candidate_contours[0], True)
        approx = cv2.approxPolyDP(candidate_contours[0], 0.02*perimeter , True)
        mask = np.zeros(cv2.cvtColor(input_image, cv2.COLOR_BGR2GRAY).shape, np.uint8)
        new_image = cv2.drawContours(mask, [approx], 0,255, -1)
        new_image = cv2.bitwise_and(input_image, input_image, mask=mask)
        (x,y) = np.where(mask==255)
        (x1, y1) = (np.min(x), np.min(y))
        (x2, y2) = (np.max(x), np.max(y))
        new_image = cv2.cvtColor(input_image, cv2.COLOR_BGR2GRAY)[x1:x2+1, y1:y2+1]
        images = new_image
    return images

input_image = cv2.imread("images/Car_1.jpg")
plate = find_plate(input_image)
show_images([plate])

